Import necessary libraries

In [1]:
# Import necessary libraries
import requests
import pandas as pd
import json
from datetime import datetime, timezone, timedelta
import time
import csv
import paho.mqtt.client as mqtt

# For Generate Unique Transaction Id for accesing API
import uuid

Import API Key

Import API Key Instructions

1. Go to the [NSW API Fuel documentation](https://api.nsw.gov.au/Product/Index/22#v-pills-doc) to get the **API Key** and **Authorization Header**.  
2. Subscribe to the Fuel API by creating an NSW API account:  
   - Create a new app with your chosen **App Name** and **App Description**.  
   - Under **Add an API Product**, select **Fuel API**.  
   - Scroll to the bottom of the page and click **Add app**.  
   - You will receive your **API Key**, **API Secret**, and **Authorization Header**.  
3. In the home directory of your assignment folder, create a file named `config_secret.py`.  
   - _Note:_ Place `config_secret.py` in the same directory as `COMP5339AS02.ipynb`.  
4. Open `config_secret.py` and add your `API_KEY` and `AuthorizationHeader`.  
   - A sample file has been provided as `config_secret_sample.py`.  

In [2]:
# grab your key from config_secret.py
API_KEY = config_secret.API_KEY
AuthorizationHeader = config_secret.AuthorizationHeader

1. Data Retrieval

In [3]:
# 1. Data Retrieval

# Fuel API Document
# https://api.nsw.gov.au/Product/Index/22#v-pills-doc

class FuelPriceCheckAPI:
    def __init__(self, API_KEY, AuthorizationHeader):
        self.url_base = "https://api.onegov.nsw.gov.au" # Base Url
        self.API_KEY = API_KEY
        self.AuthorizationHeader = AuthorizationHeader

    
    def get_datetime_now():
        '''
        Get current datetime in the API required format.
        '''

        return datetime.now(timezone.utc).strftime("%d/%m/%Y %I:%M:%S %p")
    

    def get_datetime_x_days_ago(x: int):
        '''
        Get x days ago datetime since current datetime in the API required format.
        '''

        return (datetime.now(timezone.utc) - timedelta(days=x)).strftime("%d/%m/%Y %I:%M:%S %p")
    

    def get_unique_transactionId():
        '''
        Get Unique Transaction Id.
        '''

        return str(uuid.uuid4())


    def get_accesstoken(self):
        '''
        The client can request an access token using only its client credentials (or other supported means of authentication) when the client is requesting access to the protected resources under its control, or those of another resource owner that have been previously arranged with the authorization server.
        '''

        # Config url api prameters for getting accesstoken
        url_subpart = "/oauth/client_credential/accesstoken"

        url = self.url_base + url_subpart

        query_getAccessToken = {"grant_type":"client_credentials"}

        headers_getAccessToken = {'Authorization': self.AuthorizationHeader}

        time.sleep(5)

        # Get the web result for accesstoken
        response = requests.get(url, headers=headers_getAccessToken, params=query_getAccessToken)

        # Get the accesstoken from the web result
        access_token = response.json()["access_token"]

        return access_token


    def getFuelPriceReference(self, modified_since_datetime=None):
        '''
        modified_since_datetime: The datetime UTC date and time of request. Format dd/MM/yyyy hh:mm:ss AM/PM. Default 1 day before current time.
        
        Meaning:
        modified_since_datetime: The if-modified-since header tells the API to return the reference data only if it has changed since the given UTC timestamp, otherwise it returns “Not Modified” to save bandwidth.
         
        Description:
        Returns lists of reference data to be used in subsequent requests. It is recommended to execute this call in a separate api client as response can be over 2 mb. This API returns data for NSW.
        '''

        # Config url api prameters for getting fuel price
        url_subpart = "/FuelCheckRefData/v1/fuel/lovs"

        url = self.url_base + url_subpart

        # Config modified_since_datetime
        if modified_since_datetime is None:
            modified_since_datetime = FuelPriceCheckAPI.get_datetime_x_days_ago(1)

        headers = {
            'Authorization': f'Bearer {self.get_accesstoken()}',
            'Content-Type': 'application/json; charset=utf-8',
            'apikey': self.API_KEY,
            'transactionid': FuelPriceCheckAPI.get_unique_transactionId(),
            'requesttimestamp': FuelPriceCheckAPI.get_datetime_now(),
            'if-modified-since': modified_since_datetime
        }

        time.sleep(5)

        response = requests.get(url, headers=headers)

        return response


    def getFuelPrice(self):
        '''
        Returns all current fuel prices for all service stations. There may be restrictions on how often this API request can be made. It is recommended to execute this call in a separate api client as response can be over 2 mb. This API returns data for NSW.
        '''
        # Config url api prameters for getting fuel price
        url_subpart = "/FuelPriceCheck/v1/fuel/prices"

        url = self.url_base + url_subpart

        headers = {
            'Authorization': f'Bearer {self.get_accesstoken()}',
            'Content-Type': 'application/json; charset=utf-8',
            'apikey': self.API_KEY,
            'transactionid': FuelPriceCheckAPI.get_unique_transactionId(),
            'requesttimestamp': FuelPriceCheckAPI.get_datetime_now()
        }

        time.sleep(5)

        response = requests.get(url, headers=headers)

        return response


    def getNewFuelPrice(self):
        '''
        Returns all new current prices that have been submitted since the last "/fuelpricecheck/v1/fuel/prices" or "/fuelpricecheck/v1/fuel/prices/new" request using the apikey on the current day. This API returns data for NSW.
        '''
        # Config url api prameters for getting fuel price
        url_subpart = "/FuelPriceCheck/v1/fuel/prices/new"

        url = self.url_base + url_subpart

        headers = {
            'Authorization': f'Bearer {self.get_accesstoken()}',
            'Content-Type': 'application/json; charset=utf-8',
            'apikey': self.API_KEY,
            'transactionid': FuelPriceCheckAPI.get_unique_transactionId(),
            'requesttimestamp': FuelPriceCheckAPI.get_datetime_now()
        }

        time.sleep(5)

        response = requests.get(url, headers=headers)

        return response
    
    


In [4]:
# Construct the class
fuelpriceAPI = FuelPriceCheckAPI(API_KEY, AuthorizationHeader)

In [ ]:
# To use the API, simply called fuelpriceAPI.getNewFuelPrice() or fuelpriceAPI.getFuelPrice() or fuelpriceAPI.getFuelPriceReference() depends on the usage
responseFromGetFuelPriceReference = fuelpriceAPI.getFuelPriceReference()

In [ ]:
print(json.dumps(responseFromGetFuelPriceReference.json(), indent=4, sort_keys=True))

In [5]:
# To use the API, simply called fuelpriceAPI.getNewFuelPrice() or fuelpriceAPI.getFuelPrice() or fuelpriceAPI.getFuelPriceReference() depends on the usage
responseFromGetFuelPrice = fuelpriceAPI.getFuelPrice()

In [6]:
# See the result
# print(json.dumps(responseFromGetFuelPrice.json(), indent=4, sort_keys=True))

In [7]:
# To use the API, simply called fuelpriceAPI.getNewFuelPrice() or fuelpriceAPI.getFuelPrice() or fuelpriceAPI.getFuelPriceReference() depends on the usage
responseFromGetNewFuelPrice = fuelpriceAPI.getNewFuelPrice()

In [8]:
# See the result
# print(json.dumps(responseFromGetNewFuelPrice.json(), indent=4, sort_keys=True))

API Calls
1. Create API class instance FuelPriceCheckAPI by entering argument API_KEY and AuthorizationHeader
2. Call API using the instance depends on usage
    - instance.getNewFuelPrice()
    - instance.getFuelPrice()
    - instance.getFuelPriceReference()

_Note:_ A FuelPriceCheckAPI instance has been created with the name fuelpriceAPI. simply called fuelpriceAPI.getNewFuelPrice() or fuelpriceAPI.getFuelPrice() or fuelpriceAPI.getFuelPriceReference() depends on the usage.

2. Data Integration and Storage

In [9]:
def clean_and_display_fuel_data(input_file="integrated_fuel_data.csv", output_file="integrated_fuel_data.csv", column_width=70):
    # Load the data
    df = pd.read_csv(input_file)

    # Check for missing values
    missing_values = df.isnull().sum()
    # print("Missing Values per Column:")
    # print(missing_values)

    # Drop rows with missing critical information
    df = df.dropna(subset=["ServiceStationName", "FuelCode", "PriceUpdatedDate", "Latitude", "Longitude", "Price"])

    # Fill missing values in non-critical columns
    df["Suburb"].fillna("Unknown", inplace=True)
    df["Postcode"].fillna("Unknown", inplace=True)

    # Convert FuelCode and Brand to categorical
    df["FuelCode"] = df["FuelCode"].astype("category")
    df["Brand"] = df["Brand"].astype("category")

    # Ensure Latitude, Longitude, and Price are numeric
    df["Latitude"] = pd.to_numeric(df["Latitude"], errors='coerce')
    df["Longitude"] = pd.to_numeric(df["Longitude"], errors='coerce')
    df["Price"] = pd.to_numeric(df["Price"], errors='coerce')

    # Remove invalid coordinates or prices
    df = df[
        (df["Price"] >= 0) &
        (df["Latitude"].between(-90, 90)) &
        (df["Longitude"].between(-180, 180))
    ]

    # Drop duplicate station-fuel-location combos
    df = df.drop_duplicates(subset=["ServiceStationName", "FuelCode", "Latitude", "Longitude"], keep="first")

    # Standardize text fields
    df["Brand"] = df["Brand"].str.title()
    df["Suburb"] = df["Suburb"].str.title()
    df["FuelCode"] = df["FuelCode"].str.upper()

    # Save cleaned data
    df.to_csv(output_file, index=False)
    # print(f"Cleaned data saved as '{output_file}'.")

    # # Print cleaned data as aligned columns
    # with open(output_file, mode="r", encoding="utf-8") as file:
    #     reader = csv.DictReader(file)
    #     headers = reader.fieldnames

    #     print("".join(h.ljust(column_width) for h in headers))
    #     print("-" * column_width * len(headers))

    #     for row in reader:
    #         print("".join(str(row.get(field, "")).ljust(column_width) for field in headers))

    # print("Cleaned DataFrame size:", df.shape)

    return df  

def fetch_and_save_fuel_data(fuelpriceAPI, output_file="integrated_fuel_data.csv", column_width=70):
    # Step 1: Fetch data from the API
    response = fuelpriceAPI.getFuelPrice()
    data = response.json()

    # Step 2: Create station mapping
    station_map = {
        station["code"]: {
            "ServiceStationName": station.get("name"),
            "Address": station.get("address"),
            "Brand": station.get("brand"),
            "Latitude": station.get("location", {}).get("latitude"),
            "Longitude": station.get("location", {}).get("longitude")
        }
        for station in data.get("stations", [])
    }

    # Step 3: Combine station info with prices
    combined_data = []
    for price_entry in data.get("prices", []):
        station_code = price_entry.get("stationcode")
        station_info = station_map.get(station_code)

        if station_info:
            full_address = station_info["Address"]
            try:
                parts = full_address.split(", ")
                suburb_postcode = parts[-1].rsplit(" ", 2)
                suburb = suburb_postcode[0]
                postcode = suburb_postcode[-1]
            except Exception:
                suburb, postcode = None, None

            combined_data.append({
                "ServiceStationName": station_info["ServiceStationName"],
                "Address": full_address,
                "Suburb": suburb,
                "Postcode": postcode,
                "Brand": station_info["Brand"],
                "FuelCode": price_entry.get("fueltype"),
                "Price": price_entry.get("price"),
                "PriceUpdatedDate": price_entry.get("lastupdated"),
                "Latitude": station_info["Latitude"],
                "Longitude": station_info["Longitude"]
            })

    # Step 4: Save to CSV
    df = pd.DataFrame(combined_data)
    df.to_csv(output_file, index=False)
    # print(f"Saved: {output_file}")

    # Step 5: Clean
    cleaned_df = clean_and_display_fuel_data()


def update_fuel_data(fuelpriceAPI, existing_file="integrated_fuel_data.csv", output_file="integrated_fuel_data.csv"):
    # Step 1: Load existing data
    existing_df = pd.read_csv(existing_file)

    # Step 2: Fetch new data
    response = fuelpriceAPI.getNewFuelPrice()
    new_data_json = response.json()

    # Step 3: Extract station mapping
    station_map = {
        station["code"]: {
            "ServiceStationName": station.get("name"),
            "Latitude": station.get("location", {}).get("latitude"),
            "Longitude": station.get("location", {}).get("longitude")
        }
        for station in new_data_json.get("stations", [])
    }

    # Step 4: Extract new fuel prices (subset of full dataset)
    new_price_rows = []
    for price in new_data_json.get("prices", []):
        station_code = price.get("stationcode")
        station_info = station_map.get(station_code)
        if station_info:
            new_price_rows.append({
                "ServiceStationName": station_info["ServiceStationName"],
                "FuelCode": price.get("fueltype"),
                "Latitude": station_info["Latitude"],
                "Longitude": station_info["Longitude"],
                "Price": price.get("price"),
                "PriceUpdatedDate": price.get("lastupdated")
            })

    new_df = pd.DataFrame(new_price_rows)

    if new_df.empty:
        print("No updates found in new data.")
        return existing_df

    # Step 5: Merge to update only matching entries
    updated_df = pd.merge(
        existing_df,
        new_df,
        on=["FuelCode", "Latitude", "Longitude"],
        how="left",
        suffixes=('', '_new')
    )

    # Step 6: Replace old values only if new ones are provided
    updated_df["Price"] = updated_df["Price_new"].combine_first(updated_df["Price"])
    updated_df["PriceUpdatedDate"] = updated_df["PriceUpdatedDate_new"].combine_first(updated_df["PriceUpdatedDate"])
    updated_df.drop(columns=["Price_new", "PriceUpdatedDate_new"], inplace=True)

    # Step 7: Save updated file
    updated_df.to_csv(output_file, index=False)
    # print(f"Updated {new_df.shape[0]} fuel price entries.")
    # print(f"Saved updated data to '{output_file}'.")

    return updated_df



In [10]:
fetch_and_save_fuel_data(fuelpriceAPI)

C:\Users\junjun\AppData\Local\Temp\ipykernel_33464\2669243978.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["Suburb"].fillna("Unknown", inplace=True)
C:\Users\junjun\AppData\Local\Temp\ipykernel_33464\2669243978.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For exa

In [11]:
updated_df = update_fuel_data(fuelpriceAPI)

# # Print cleaned data as aligned columns
# with open('integrated_fuel_data.csv', mode="r", encoding="utf-8") as file:
#     reader = csv.DictReader(file)
#     headers = reader.fieldnames

#     print("".join(h.ljust(70) for h in headers))
#     print("-" * 70 * len(headers))

#     for row in reader:
#         print("".join(str(row.get(field, "")).ljust(70) for field in headers))

No updates found in new data.


3. Data Publishing via MQTT

In [12]:
# 3. Data Publishing via MQTT

# Publisher


In [15]:
# Define callbacks
def on_connect(client, userdata, connect_flags, reason_code, properties):
    print("Connected with result code", str(reason_code))
    # Subscribe as soon as we connect
    client.subscribe("COMP5339/Assignment02/Group07/FuelPrice")

def on_message(client, userdata, msg):
    print(json.loads(msg.payload))

def publish_data():
    # 1. Retrieve cleaned data
    clean_data = pd.read_csv("integrated_fuel_data.csv")
    
    # 2. Create client and attach callbacks
    client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
    client.on_connect = on_connect
    client.on_message = on_message
    
    # 3. Connect to broker
    client.connect("broker.hivemq.com", 1883, keepalive=60)
    
    # 4. Start the network loop in a background thread
    client.loop_start()
    
    # 5. Publish periodically
    for row in clean_data.itertuples():
    
        data = {
            "Index": row.Index, 
            "ServiceStationName": row.ServiceStationName,
            "Address": row.Address, 
            "Suburb": row.Suburb, 
            "Postcode": row.Postcode, 
            "Brand": row.Brand, 
            "FuelCode": row.FuelCode, 
            "Price": row.Price, 
            "PriceUpdatedDate": row.PriceUpdatedDate, 
            "Latitude": row.Latitude, 
            "Longitude": row.Longitude
        }
    
        client.publish("COMP5339/Assignment02/Group07/FuelPrice", json.dumps(data), qos=2)
    
        time.sleep(0.1)
    
    # 6. Clean up
    client.loop_stop()
    client.disconnect()

In [23]:
publish_data()

Connected with result code Success
{'Index': 3, 'ServiceStationName': 'Coles Express Coffs Harbour', 'Address': '208-212 Pacific Hwy North, Coffs Harbour NSW 2450', 'Suburb': 'Coffs Harbour', 'Postcode': '2450', 'Brand': 'Coles Express', 'FuelCode': 'P98', 'Price': 203.9, 'PriceUpdatedDate': '12/04/2025 06:15:45', 'Latitude': -30.286083, 'Longitude': 153.124146}
{'Index': 4, 'ServiceStationName': 'Coles Express Coffs Harbour', 'Address': '208-212 Pacific Hwy North, Coffs Harbour NSW 2450', 'Suburb': 'Coffs Harbour', 'Postcode': '2450', 'Brand': 'Coles Express', 'FuelCode': 'U91', 'Price': 180.9, 'PriceUpdatedDate': '12/04/2025 06:15:45', 'Latitude': -30.286083, 'Longitude': 153.124146}
{'Index': 5, 'ServiceStationName': 'Shell Reddy  Express Glendale', 'Address': '593 Main Rd, Glendale NSW 2285', 'Suburb': 'Glendale', 'Postcode': '2285', 'Brand': 'Reddy Express', 'FuelCode': 'DL', 'Price': 186.9, 'PriceUpdatedDate': '16/05/2025 03:15:47', 'Latitude': -32.927065, 'Longitude': 151.637938

KeyboardInterrupt: 

{'Index': 692, 'ServiceStationName': '7-Eleven Castle Hill', 'Address': '255 -257 Old Northern Road, Castle Hill NSW 2154', 'Suburb': 'Castle Hill', 'Postcode': '2154', 'Brand': '7-Eleven', 'FuelCode': 'P98', 'Price': 182.9, 'PriceUpdatedDate': '22/05/2025 20:14:19', 'Latitude': -33.734803, 'Longitude': 151.003582}
{'Index': 693, 'ServiceStationName': '7-Eleven Castle Hill', 'Address': '255 -257 Old Northern Road, Castle Hill NSW 2154', 'Suburb': 'Castle Hill', 'Postcode': '2154', 'Brand': '7-Eleven', 'FuelCode': 'PDL', 'Price': 189.9, 'PriceUpdatedDate': '12/05/2025 20:44:21', 'Latitude': -33.734803, 'Longitude': 151.003582}


4. Data Subscribing and Visualisation

In [ ]:
# 4. Data Subscribing and Visualisation

5. Continuous Execution

In [ ]:
# 5. Continuous Execution